In [48]:
import numpy as np
import tensorflow as tf

In [49]:
words = open('names.txt', 'r').read().splitlines()

In [50]:
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [51]:
print(f'min = {min(len(w) for w in words)}, max = {max(len(w) for w in words)}')

min = 2, max = 15


In [52]:
alphabet = sorted(list(set(''.join(words))))

stoi = {s: i for i, s in enumerate(('.', *alphabet))}
itos = {i: s for s, i in stoi.items()}
num_classes = len(stoi)

In [53]:
xs, ys = [], []

for w in words:
    chars = ['.'] + list(w) + ['.']
    for c_curr, c_next in zip(chars, chars[1:]):
        #print(c_curr, c_next)
        xs.append(stoi[c_curr])
        ys.append(stoi[c_next])

xs = tf.convert_to_tensor(xs)
ys = tf.convert_to_tensor(ys)


In [54]:
xenc = tf.one_hot(xs, num_classes)
yidx = tf.stack([tf.range(ys.shape[0]), ys], axis=1)

In [55]:
W = tf.Variable(tf.random.normal((num_classes, num_classes), seed=1), name='W', trainable=True)

In [73]:
learning_rate = 20
reg_strength = 0.1

for _ in range(10000):

    with tf.GradientTape() as tape:
        logits = xenc @ W
        counts = tf.math.exp(logits)
        probs = counts / tf.math.reduce_sum(counts, axis=1, keepdims=True)
        loss = tf.math.reduce_mean(-tf.math.log(tf.gather_nd(probs, yidx))) + reg_strength * tf.math.reduce_mean(W**2)

    print(loss.numpy())

    dl_dw = tape.gradient(loss, W)
    W.assign_sub(learning_rate*dl_dw)


2.5863626
2.5863626
2.5863626
2.5863626
2.5863626
2.5863624
2.5863624
2.5863626
2.5863624
2.5863624
2.5863626
2.5863624
2.5863624
2.5863626
2.5863626
2.5863626
2.5863626
2.5863626
2.5863626
2.5863624
2.5863624
2.5863624
2.5863626
2.5863624
2.5863628
2.5863626
2.5863628
2.5863626
2.5863624
2.5863626
2.5863626
2.5863626
2.5863624
2.5863626
2.5863626
2.5863626
2.5863624
2.5863626
2.5863624
2.5863624
2.5863628
2.5863626
2.5863628
2.5863626
2.5863626
2.5863626
2.5863626
2.5863624
2.5863626
2.5863626
2.5863624
2.5863626
2.5863624
2.5863626
2.5863624
2.5863624
2.5863626
2.5863624
2.5863624
2.5863624
2.5863626
2.5863626
2.5863626
2.5863626
2.5863626
2.5863624
2.5863626
2.5863624
2.5863624
2.5863626
2.5863624
2.5863624
2.5863626
2.5863624
2.5863624
2.5863624
2.5863624
2.5863624
2.5863624
2.5863624
2.5863624
2.5863624
2.5863621
2.5863621
2.5863626
2.5863626
2.5863626
2.5863626
2.5863626
2.5863624
2.5863624
2.5863626
2.5863624
2.5863626
2.5863626
2.5863626
2.5863624
2.5863624
2.5863626
2.5863626


KeyboardInterrupt: 